In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

def analyze_real_estate_data(file_path, enable_visuals=True, pairplot_vars=None):
    """
    Analyzuje data o cenách bytů v Praze, provádí shlukovou analýzu a predikuje ceny.
    :param file_path: Cesta k CSV souboru obsahujícímu vstupní data.
    :param enable_visuals: Ovládání vizualizací pro rychlejší běh v Excelu.
    """
    # 1️⃣ Načtení dat
    df = pd.read_excel(file_path, parse_dates=['Datum podání'])
    # dayfirst argument is deprecated, use dayfirst within to_datetime
    df['Datum podání'] = pd.to_datetime(df['Datum podání'], dayfirst=True)
    df['Datum podání (dny)'] = (df['Datum podání'] - df['Datum podání'].min()).dt.days

    # Výběr relevantních sloupců
    features = ['LAT', 'LON', 'JC byty [Kč/m2]', 'Plocha [m2]', 'Datum podání (dny)']
    df = df.dropna(subset=features)

    # 2️⃣ Normalizace dat pro K-means
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[features])

    # 3️⃣ Shluková analýza (K-means)
    k = 4
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=5, max_iter=300)
    df['Cluster'] = kmeans.fit_predict(X_scaled)

    if enable_visuals:
        # 4️⃣ Vizualizace shluků podle polohy
        plt.figure(figsize=(10, 6))
        plt.scatter(df['LAT'], df['LON'], c=df['Cluster'], cmap='viridis', marker='o')
        plt.xlabel('Latitude')
        plt.ylabel('Longitude')
        plt.title('K-means shluková analýza prodaných bytů')
        plt.colorbar(label='Cluster')
        plt.grid(True)
        plt.savefig("shlukova_analyza.png")
        plt.show()

        # 5️⃣ Korelační analýza
        plt.figure(figsize=(8, 6))
        sns.heatmap(df[features].corr(), annot=True, cmap='coolwarm', fmt='.2f')
        plt.title('Korelační matice mezi faktory')
        plt.savefig("korelacni_matrice.png")
        plt.show()

        # 6️⃣ Párový graf pro shlukování (omezený vzorek pro rychlost)
        sampled_df = df.sample(n=min(5000, len(df)), random_state=42)

        # Vykreslení pouze zadaných sloupců v pairplot, including 'Cluster' column
        plot_vars = pairplot_vars if pairplot_vars else sampled_df.columns
        # Ensure 'Cluster' is in plot_vars for hue to work
        plot_vars = list(plot_vars) + ['Cluster'] if 'Cluster' not in plot_vars else plot_vars
        sns.pairplot(sampled_df[plot_vars], hue='Cluster', diag_kind='kde', palette='viridis')


        plt.suptitle('Párový graf shlukování bytů', fontsize=16)
        plt.savefig("parovy_graf_shlukovani.png")
        plt.show()

    # 7️⃣ Predikce cen bytů
    X = df[['LAT', 'LON', 'Plocha [m2]', 'Datum podání (dny)']]
    y = df['JC byty [Kč/m2]']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model 1: Lineární regrese
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    y_pred_lr = lin_reg.predict(X_test)
    r2_lr = r2_score(y_test, y_pred_lr)

    # Model 2: Random Forest
    rf_reg = RandomForestRegressor(n_estimators=50, random_state=42)
    rf_reg.fit(X_train, y_train)
    y_pred_rf = rf_reg.predict(X_test)
    r2_rf = r2_score(y_test, y_pred_rf)

    print("\n📊 Výsledky modelů:")
    print(f"Lineární regrese: R² = {r2_lr:.3f}, MSE = {mean_squared_error(y_test, y_pred_lr):.2f}")
    print(f"Random Forest: R² = {r2_rf:.3f}, MSE = {mean_squared_error(y_test, y_pred_rf):.2f}")

    if enable_visuals:
        # 8️⃣ Grafy závislostí faktorů na ceně včetně R²
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        factors = ['LAT', 'Plocha [m2]', 'Datum podání (dny)']

        for i, factor in enumerate(factors):
            sns.regplot(ax=axes[i], x=df[factor], y=df['JC byty [Kč/m2]'], line_kws={'color': 'red'})
            r2 = r2_score(df['JC byty [Kč/m2]'], LinearRegression().fit(df[[factor]], df['JC byty [Kč/m2]']).predict(df[[factor]]))
            axes[i].set_title(f'Závislost {factor} na ceně bytu (R²={r2:.2f})')
            axes[i].set_xlabel(factor)
            axes[i].set_ylabel('JC byty [Kč/m2]')

        plt.suptitle('Analýza faktorů ovlivňujících cenu bytu', fontsize=16)
        plt.tight_layout()
        plt.savefig("zavislosti_faktoru.png")
        plt.show()

    return df

# 📌 Použití funkce s daty ze souboru
vybrane_sloupce = ['JC byty [Kč/m2]', 'LAT', 'LON', 'Vzdálenost [km]', 'Plocha [m2]', 'Datum podání (dny)']
df = analyze_real_estate_data("valuo test data.xlsx", enable_visuals=True, pairplot_vars=vybrane_sloupce)


In [23]:
%load_ext sql
import urllib.parse

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=reports;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = "mssql+pyodbc:///?odbc_connect=%s" % params

# Nastavení připojení
%sql $connection_url

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
from sqlalchemy import create_engine
import pandas as pd

server = 'localhost'
database = 'reports'
conn_str = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"

engine = create_engine(conn_str)

# Ověření spojení
with engine.connect() as conn:
    df = pd.read_sql("SELECT TOP 10 * FROM [reports].[dbo].[revolut_stocks]", conn)

df.head()


,ID,timestamp,date_nvchar,date,ticker,type,quantity,price,amount,currency,fx_rate
0,1,2025-01-17 10:21:50,2021-10-23T12:37:44.343764Z,2021-10-23,,CASH TOP-UP,NaN,NaN,100.0,USD,0.0455
1,2,2025-01-17 10:21:50,2021-10-25T13:30:37.704257Z,2021-10-25,AAPL,BUY - MARKET,0.259843,149.09,40.0,USD,0.0452
2,3,2025-01-17 10:21:50,2021-10-25T13:30:54.743019Z,2021-10-25,TSLA,BUY - MARKET,0.052744,947.97,50.0,USD,0.0452
3,4,2025-01-17 10:21:50,2021-10-26T05:15:11.553853Z,2021-10-26,,CASH TOP-UP,NaN,NaN,60.0,USD,0.0451
4,5,2025-01-17 10:21:50,"""2021-10-26T13:30:17.113904Z",2021-10-26,AMZN,BUY - MARKET,0.008574,3352.00,30.0,USD,0.0452


In [3]:
query = """
SELECT 
    YEAR(date) as _year,
    FORMAT(ROUND(SUM(amount),2), '0.###') as value$,
    COUNT(amount) as payouts
FROM [reports].[dbo].[revolut_stocks]
WHERE type = 'DIVIDEND'
GROUP BY YEAR(date)
ORDER BY _year DESC;
"""

df = pd.read_sql(query, engine)
df.head()





,_year,value$,payouts
0,2025,10.49,6
1,2024,86.76,111
2,2023,32.39,60
3,2022,6.76,21
4,2021,0.12,2


In [4]:
%%sql
SELECT 1 AS test;


 * mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Dlocalhost%3BDatabase%3Dreports%3BTrusted_Connection%3Dyes
Done.


test
1


In [15]:

%%sql
-- Prehled vyplacenych dividend po letech
select 
        
	    year(date) as _year
	   ,format(round(sum(amount),2), '0.###') as value$     -- format prevede numeric na text
	   ,count(amount) as payouts
from [reports].[dbo].[revolut_stocks]
where 
     type = 'DIVIDEND'
group by  year(date)
order by _year desc;


 * mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Dlocalhost%3BDatabase%3Dreports%3BTrusted_Connection%3Dyes
Done.


_year,value$,payouts
2025,10.49,6
2024,86.76,111
2023,32.39,60
2022,6.76,21
2021,0.12,2
